In [ ]:
pip install pandas requests


Obtener Noticias

In [30]:
import requests
import pandas as pd

# Clave de API (Sustituye con tu clave real)
NEWS_API_KEY = "e624185635c04d0ca296acd2724015c6"

# Función para obtener noticias de un país
def obtener_noticias(pais="us"):
    try:
        url = f"https://newsapi.org/v2/top-headlines?country={pais}&apiKey={NEWS_API_KEY}"
        response = requests.get(url)
        data = response.json()

        if "articles" not in data:
            print(f"⚠️ No se encontraron noticias para {pais}.")
            return pd.DataFrame()

        noticias = [{
            "pais": pais.upper(),
            "fecha": article.get("publishedAt", "Desconocido"),
            "titulo": article.get("title", "Sin título"),
            "fuente": article["source"].get("name", "Desconocido")
        } for article in data.get("articles", [])[:10]]  # Se toman las primeras 10 noticias

        return pd.DataFrame(noticias)
    except Exception as e:
        print(f"❌ Error obteniendo noticias para {pais}: {e}")
        return pd.DataFrame()

# Obtener datasets de noticias de EE.UU. y Perú
df_usa = obtener_noticias("us")
df_peru = obtener_noticias("pe")

# Combinar datasets
df_noticias = pd.concat([df_usa, df_peru], ignore_index=True)

# 🔹 **Verificar si hay datos antes de calcular indicadores**
if df_noticias.empty:
    print("⚠️ No hay datos disponibles. No se pueden calcular indicadores de calidad.")
else:
    # ✅ **Convertir la columna "fecha" a datetime y eliminar la zona horaria**
    df_noticias['fecha'] = pd.to_datetime(df_noticias['fecha'], errors='coerce')
    df_noticias['fecha'] = df_noticias['fecha'].dt.tz_localize(None)  # Eliminar la zona horaria

    # ❌ Eliminar filas con fecha nula para evitar errores
    df_noticias = df_noticias.dropna(subset=['fecha'])

    # 1️⃣ **Completitud**: % de noticias con título y fuente
    completitud_titulo = 100 - (df_noticias['titulo'].isna().mean() * 100)
    completitud_fuente = 100 - (df_noticias['fuente'].isna().mean() * 100)

    # 2️⃣ **Diversidad de fuentes**: Número de fuentes únicas
    fuentes_unicas = df_noticias['fuente'].nunique()

    # 3️⃣ **Recencia de datos**: Noticias publicadas en las últimas 24 horas
    fecha_limite = pd.Timestamp.now().tz_localize(None) - pd.Timedelta(days=1)
    recientes = (df_noticias['fecha'] >= fecha_limite).mean() * 100

    # 4️⃣ **Noticias sin título "Sin título"**: % de noticias sin título real
    sin_titulo_real = (df_noticias['titulo'] == "Sin título").mean() * 100

    # Mostrar resultados
    print("\n📊 Indicadores de Calidad de Datos:")
    print(f"✅ Completitud (Título): {completitud_titulo:.2f}% de noticias tienen título")
    print(f"✅ Completitud (Fuente): {completitud_fuente:.2f}% de noticias tienen fuente")
    print(f"✅ Diversidad de fuentes: {fuentes_unicas} fuentes únicas")
    print(f"✅ Recencia: {recientes:.2f}% de noticias publicadas en las últimas 24 horas")
    print(f"⚠️ Noticias sin título real: {sin_titulo_real:.2f}% de los casos")

    # Guardar en CSV
    df_noticias.to_csv("noticias.csv", index=False)

    # Mostrar primeros registros
    print("\n📢 Noticias obtenidas:\n", df_noticias.head())



📊 Indicadores de Calidad de Datos:
✅ Completitud (Título): 100.00% de noticias tienen título
✅ Completitud (Fuente): 100.00% de noticias tienen fuente
✅ Diversidad de fuentes: 7 fuentes únicas
✅ Recencia: 0.00% de noticias publicadas en las últimas 24 horas
⚠️ Noticias sin título real: 0.00% de los casos

📢 Noticias obtenidas:
   pais               fecha                                             titulo  \
0   US 2025-03-27 23:22:00  Judge orders Trump administration to keep Sign...   
1   US 2025-03-27 23:13:58  Trump tariffs: Can Vietnam golf its way out of...   
2   US 2025-03-27 23:07:16  'We're going to feel the impacts': NC Health D...   
3   US 2025-03-27 22:57:32  Selena's Killer Yolanda Saldívar Denied Parole...   
4   US 2025-03-27 21:44:00  Republicans grow concerned about Trump’s handp...   

     fuente  
0       CNN  
1  BBC News  
2  WRAL.com  
3    PEOPLE  
4       CNN  


Obtener Temperaturas

In [25]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Clave de API
WEATHER_API_KEY = "6215ab1921e5d5625a96fd0df204c2cf"

# Función para obtener temperatura diaria en 2025
def obtener_temperaturas_anuales(ciudad="Lima"):
    try:
        fecha_actual = datetime(2025, 1, 1)
        temperaturas = []

        while fecha_actual.year == 2025:
            fecha_str = fecha_actual.strftime("%Y-%m-%d")
            url = f"http://api.openweathermap.org/data/2.5/weather?q={ciudad}&units=metric&appid={WEATHER_API_KEY}"
            response = requests.get(url)
            data = response.json()

            temperatura = data.get("main", {}).get("temp", "No disponible")
            temperaturas.append({"ciudad": ciudad, "fecha": fecha_str, "temperatura_c": temperatura})

            fecha_actual += timedelta(days=1)  # Avanzar un día

        return pd.DataFrame(temperaturas)
    except Exception as e:
        print(f"Error obteniendo temperaturas: {e}")
        return pd.DataFrame()

# Obtener dataset
df_temperaturas = obtener_temperaturas_anuales("Lima")

# 🔹 Indicadores de calidad de datos

# 1. **Completitud**: % de datos disponibles vs. esperados
completitud = 100 - (df_temperaturas['temperatura_c'].isna().mean() * 100)

# 2. **Consistencia temporal**: % de días con datos (esperado 365 días)
dias_esperados = 365
dias_reales = df_temperaturas['fecha'].nunique()
consistencia = (dias_reales / dias_esperados) * 100

# 3. **Validez**: Temperaturas dentro del rango esperado en Lima (-5°C a 40°C)
df_temperaturas['temperatura_c'] = pd.to_numeric(df_temperaturas['temperatura_c'], errors='coerce')
fuera_de_rango = ((df_temperaturas['temperatura_c'] < -5) | (df_temperaturas['temperatura_c'] > 40)).mean() * 100

# 4. **Precisión decimal**: Verificar que los valores tienen 1 o 2 decimales
precision_decimal = df_temperaturas['temperatura_c'].dropna().astype(str).apply(lambda x: len(x.split(".")[-1]) if "." in x else 0)
precision_correcta = ((precision_decimal == 1) | (precision_decimal == 2)).mean() * 100

# 5. **Errores de API**: % de datos "No disponible"
errores_api = (df_temperaturas['temperatura_c'].isna().mean()) * 100

# Mostrar resultados
print("\n📊 Indicadores de Calidad de Datos:")
print(f"✅ Completitud: {completitud:.2f}% de datos disponibles")
print(f"✅ Consistencia: {consistencia:.2f}% de días con datos registrados")
print(f"⚠️ Validez: {fuera_de_rango:.2f}% de valores fuera del rango esperado (-5°C a 40°C)")
print(f"✅ Precisión decimal: {precision_correcta:.2f}% con 1 o 2 decimales")
print(f"⚠️ Errores de API: {errores_api:.2f}% de respuestas incorrectas o vacías")

# Guardar en CSV
df_temperaturas.to_csv("temperaturas_2025.csv", index=False)

# Mostrar primeros registros
print("\nTemperaturas:\n", df_temperaturas.head())



📊 Indicadores de Calidad de Datos:
✅ Completitud: 100.00% de datos disponibles
✅ Consistencia: 100.00% de días con datos registrados
⚠️ Validez: 0.00% de valores fuera del rango esperado (-5°C a 40°C)
✅ Precisión decimal: 100.00% con 1 o 2 decimales
⚠️ Errores de API: 0.00% de respuestas incorrectas o vacías

Temperaturas:
   ciudad       fecha  temperatura_c
0   Lima  2025-01-01          24.39
1   Lima  2025-01-02          24.39
2   Lima  2025-01-03          24.39
3   Lima  2025-01-04          24.39
4   Lima  2025-01-05          24.39


Obtener Tipo Cambio

In [24]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Claves de API
NEWS_API_KEY = "be17d72a88da4150a7358d5f2512dc25"
WEATHER_API_KEY = "6215ab1921e5d5625a96fd0df204c2cf"
EXCHANGE_API_KEY = "5qUu0OStUqC30001qs7mJyiamFHElBmz"

# Función para obtener tipo de cambio diario durante dos años (2024 y 2025)
def obtener_tipo_cambio_anual(base="USD", destino="PEN", anios=2):
    try:
        fecha_actual = datetime(2024, 1, 1)  # Ahora comienza en 2024
        fecha_final = fecha_actual + timedelta(days=365 * anios)
        tipos_cambio = []

        while fecha_actual < fecha_final:
            fecha_str = fecha_actual.strftime("%Y-%m-%d")
            url = f"https://api.exchangerate-api.com/v4/latest/{base}"
            response = requests.get(url)
            data = response.json()

            tipo_cambio = data.get("rates", {}).get(destino, "No disponible")
            tipos_cambio.append({"fecha": fecha_str, "moneda": f"{base}/{destino}", "tipo_cambio": tipo_cambio})

            fecha_actual += timedelta(days=1)  # Avanzar un día

        return pd.DataFrame(tipos_cambio)
    except Exception as e:
        print(f"Error obteniendo tipo de cambio: {e}")
        return pd.DataFrame()

# Obtener dataset para 2024 y 2025
df_tipo_cambio = obtener_tipo_cambio_anual("USD", "PEN", anios=2)

# 🔹 Indicadores de calidad de datos
# 1. Completitud de datos
completitud = 100 - (df_tipo_cambio['tipo_cambio'].isna().mean() * 100)

# 2. Consistencia temporal
dias_esperados = 365 * 2  # 2024 y 2025
dias_reales = df_tipo_cambio['fecha'].nunique()
consistencia = (dias_reales / dias_esperados) * 100

# 3. Validez del tipo de cambio (fuera del rango esperado 3-5)
df_tipo_cambio['tipo_cambio'] = pd.to_numeric(df_tipo_cambio['tipo_cambio'], errors='coerce')
fuera_de_rango = ((df_tipo_cambio['tipo_cambio'] < 3) | (df_tipo_cambio['tipo_cambio'] > 5)).mean() * 100

# 4. Precisión decimal (4 decimales)
precision_decimal = df_tipo_cambio['tipo_cambio'].dropna().astype(str).apply(lambda x: len(x.split(".")[-1]) if "." in x else 0)
precision_correcta = (precision_decimal == 4).mean() * 100

# 5. Errores en la API
errores_api = (df_tipo_cambio['tipo_cambio'].isna().mean()) * 100

# Mostrar resultados
print("\n📊 Indicadores de Calidad de Datos para 2024 y 2025:")
print(f"✅ Completitud: {completitud:.2f}% de datos disponibles")
print(f"✅ Consistencia: {consistencia:.2f}% de días con datos registrados")
print(f"⚠️ Validez: {fuera_de_rango:.2f}% de valores fuera del rango esperado (3-5)")
print(f"✅ Precisión decimal: {precision_correcta:.2f}% con 4 decimales")
print(f"⚠️ Errores de API: {errores_api:.2f}% de respuestas incorrectas o vacías")

# Guardar en CSV
df_tipo_cambio.to_csv("tipo_cambio_2024_2025.csv", index=False)

# Mostrar primeros registros
print("\nTipo de Cambio:\n", df_tipo_cambio.head())



📊 Indicadores de Calidad de Datos para 2024 y 2025:
✅ Completitud: 100.00% de datos disponibles
✅ Consistencia: 100.00% de días con datos registrados
⚠️ Validez: 0.00% de valores fuera del rango esperado (3-5)
✅ Precisión decimal: 0.00% con 4 decimales
⚠️ Errores de API: 26.03% de respuestas incorrectas o vacías

Tipo de Cambio:
         fecha   moneda  tipo_cambio
0  2024-01-01  USD/PEN         3.64
1  2024-01-02  USD/PEN         3.64
2  2024-01-03  USD/PEN         3.64
3  2024-01-04  USD/PEN         3.64
4  2024-01-05  USD/PEN         3.64
